In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


from sklearn import preprocessing
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

In [90]:
# Load the entire dataset into a pandas dataframe

df = pd.read_parquet('/nobackup/amimalik/bits/dataset/dimensions/4Q/working_dataset.parquet')
model_df = pd.read_csv('/nobackup/amimalik/bits/dataset/dimensions/4Q/sorted_failures.csv')

In [91]:
def get_model_data(df, model):
    model_df = df[df['model'] == model]
    model_df = model_df.drop(columns=['model'])
    return model_df.drop(columns=['serial_number'])

In [92]:
# to be enabled later for all datasets and models
for models in model_df['model']:
    final_df = get_model_data(df, models)

In [93]:
final_df = get_model_data(df, 'ST12000NM0008')

In [94]:
final_df = final_df.drop(columns=['I/O_Error_Detection_and_Correction'])

In [95]:
print(np.isnan(final_df).sum())

failure                          0
Reallocated_Sectors_Count        2
Power-On_Hours                   2
Reported_Uncorrectable_Errors    2
Load_Unload_Cycle                2
Current_Pending_Sector_Count     2
Offline_Uncorrectable            2
Head_Flying_Hours                2
Total_LBAs_Written               2
Total_LBAs_Read                  2
dtype: int64


In [96]:
final_df.fillna(final_df.mean(), inplace=True)

In [97]:
final_df.dropna(axis =1,inplace=True)

In [98]:
final_df.shape

(53036, 10)

In [106]:
# Define a function that performs stratified cross-validation

def stratified_data_split(df):
    # Define the number of folds
    n_folds = 5

    # Create an instance of StratifiedKFold
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

    # Split the data into X and y
    X = df.drop(columns=['failure'])
    y = df['failure']
    
    # Perform stratified cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        y_train = np.array(y_train).reshape(-1, 1)
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))


        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        yield X_train, y_train, X_test, y_test

In [100]:
stratified_data_split(final_df)

<generator object stratified_data_split at 0x7f37128a1580>

In [107]:


def model_fit(X_train, y_train, x_test, y_test, epochs=10, batch_size=32, learning_rate=0.001):
    # Define the model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1,9)))
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    
    # Fit the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    
    return model

In [109]:
for train_X, train_y, test_X, test_y in stratified_data_split(final_df):
        #print(train_X.shape, train_y.shape, test_X.shape)
        #print(train_X.shape[0], train_X.shape[1])
        #print(np.isnan(train_X).sum())
        #print(np.isnan(train_y).sum())

        #scaler = StandardScaler()
        #X_train_scaled = scaler.fit_transform(train_X)
        #X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))

        #print(X_train_scaled.shape, train_y.shape, test_X.shape)

        model_fit(train_X, train_y, test_X, test_y, epochs=10, batch_size=32, learning_rate=0.001)

(42428, 1, 9) (42428, 1) (10608, 9) (10608,)
Epoch 1/10


/ws/amimalik-bgl/.pyenv/versions/3.11.4/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1326/1326 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0032
Epoch 2/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0033
Epoch 3/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 950us/step - loss: 0.0037
Epoch 4/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step - loss: 0.0032
Epoch 5/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 929us/step - loss: 0.0033
Epoch 6/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 967us/step - loss: 0.0030
Epoch 7/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 956us/step - loss: 0.0031
Epoch 8/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 963us/step - loss: 0.0034
Epoch 9/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - loss: 0.0033
Epoch 10/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - loss: 0.0034
(42429, 1, 9) (42429, 1) (10607, 9) (10607,)
Epoch 1/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 2s 978us/step - loss: 0.0035
Epoch 2/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 920us/step - loss: 0.0033
Epoch 3/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 951us/step - loss: 0.0037
Epoch 4/10
1326/1326 ━━━━━━━━━━━━━━━━━━━━ 1s 9

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))